In [ ]:
pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json


{'kaggle.json': b'{"username":"lukewu0917","key":"0d7910739683ff5bddb6bae3293f992f"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d soumikrakshit/anime-faces

 98% 433M/441M [00:02<00:00, 157MB/s]
100% 441M/441M [00:02<00:00, 155MB/s]


In [ ]:
!unzip anime-faces.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/data/5499.png      
  inflating: data/data/55.png        
  inflating: data/data/550.png       
  inflating: data/data/5500.png      
  inflating: data/data/5501.png      
  inflating: data/data/5502.png      
  inflating: data/data/5503.png      
  inflating: data/data/5504.png      
  inflating: data/data/5505.png      
  inflating: data/data/5506.png      
  inflating: data/data/5507.png      
  inflating: data/data/5508.png      
  inflating: data/data/5509.png      
  inflating: data/data/551.png       
  inflating: data/data/5510.png      
  inflating: data/data/5511.png      
  inflating: data/data/5512.png      
  inflating: data/data/5513.png      
  inflating: data/data/5514.png      
  inflating: data/data/5515.png      
  inflating: data/data/5516.png      
  inflating: data/data/5517.png      
  inflating: data/data/5518.png      
  inflating: data/data/5519.png      
  inflating: data/data/552.png       

In [ ]:
!rm /content/anime-faces.zip

In [ ]:
!rm -rf /content/data/data

In [ ]:
import tensorflow as tf
import cv2
import numpy
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import random
from PIL import Image
import os

!pip install visualkeras
!pip uninstall gan
!pip install gan
import visualkeras
# from gan import GAN

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Found existing installation: gan 0.0.1
Uninstalling gan-0.0.1:
  Would remove:
    /usr/local/bin/gan
    /usr/local/lib/python3.9/dist-packages/gan-0.0.1.dist-info/*
    /usr/local/lib/python3.9/dist-packages/gan/*
    /usr/local/lib/python3.9/dist-packages/package_name/*
Proceed (Y/n)? Y
  Successfully uninstalled gan-0.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gan-0.0.1-py3-none-any.whl (5.5 kB)


In [ ]:
class Plots:
    def __init__(self):
        pass

    @staticmethod
    def plot_images_from_array(data=None, labels=None, rows=2, columns=2, figsize=(6, 6), randomize=False, update_plot=False):
        if data is None:
            return None

        fig, axs = plt.subplots(rows, columns, sharey='none', tight_layout=True, figsize=figsize, facecolor='white')

        n = 0
        for i in range(rows):
            for j in range(columns):
                if randomize:
                    axs[i, j].imshow(data[random.choice(range(len(data)-1))], cmap='gray')
                    if labels is not None: axs[i, j].set(title=labels[n])
                else:
                    axs[i, j].imshow(data[i+(j*rows)], cmap='gray')
                    if labels is not None: axs[i, j].set(title=labels[n])
                axs[i, j].axis('off')
                n = n + 1
        plt.show()

    @staticmethod
    def plot_images_from_directory(path=None, rows=2, columns=2, figsize=(6, 6), randomize=False, update_plot=False):
        if path is None:
            return None

        if path[-1] == '/':
            path_copy = path[:-1]
        else:
            path_copy = path

        file_names = [path_copy + '/' + i for i in os.listdir(path_copy)]

        fig, axs = plt.subplots(rows, columns, sharey='none', tight_layout=True, figsize=figsize, facecolor='white')

        for i in range(rows):
            for j in range(columns):
                if randomize:
                    image = Image.open(file_names[random.choice(range(len(file_names)-1))])
                    axs[i, j].imshow(image, cmap='gray')
                else:
                    image = Image.open(file_names[i+(j*rows)])
                    axs[i, j].imshow(image)
                axs[i, j].axis('off')
        plt.show()

    @staticmethod
    def plot_images_from_folder(path=None, num_images=5, figsize=(6, 6), randomize=False, update_plot=False):
        if path is None:
            return None

        if path[-1] == '/':
            path_copy = path[:-1]
        else:
            path_copy = path

        dir_names = [path_copy + '/' + i for i in os.listdir(path_copy)]

        fig, axs = plt.subplots(len(dir_names), num_images, sharey='none', tight_layout=True, figsize=figsize,
                                facecolor='white')

        for i in range(len(dir_names)):
            temp_path = dir_names[i] + '/'
            for j in range(num_images):
                if randomize:
                    image = Image.open(temp_path + random.choice(os.listdir(temp_path)))
                    axs[i, j].imshow(image)
                else:
                    image = Image.open(temp_path + os.listdir(temp_path)[j])
                    axs[i, j].imshow(image)
                axs[i, j].axis('off')
        plt.show()



class PreprocessData:
    def __init__(self):
        pass

    @staticmethod
    def load_image(path, resize_height=64, resize_width=64, d_type=False):
        image = tf.io.read_file(path)
        image = tf.io.decode_png(image)
        image = tf.image.resize(image, (resize_height, resize_width))
        if d_type:
            image = tf.cast(image, d_type)
        return image

    @staticmethod
    def normalize_values(x, to_val=(0, 1), from_val=(0, 255), d_type=False):
        if d_type:
            x = tf.cast(x, d_type)

        return (x - from_val[0]) * (to_val[1] - to_val[0]) / (from_val[1] - from_val[0]) + to_val[0]



plots = Plots()


In [ ]:
class Utils():
  def __init__(self):
    self.IMGAE_DIM = (64, 64, 3)
    pass

  def plot_images(data=None, labels=None, rows=2, columns=2, figsize=(12,6), randomize=False):
    if data is None:
      return None

    fig, axs = plt.subplots(rows, columns, sharey=False, tight_layout=True, figsize=figsize, facecolor='white')

    n=0
    for i in range(rows):
      for j in range(columns):
        if randomize:
          n = random.choice(range(len(data)))
          axs[i,j].matshow(data[n], cmap='gray')
          if labels is not None: axs[i,j].set(title=labels[n])
        else:
          axs[i,j].matshow(data[n], cmap='gray')
          if labels is not None: axs[i,j].set(title=labels[n])
        axs[i,j].axis('off')
        n=n+1
    plt.show()

  def normalize_values(x, to_val=(0, 1), from_val=(0, 255), dtype=False):
    if dtype != False:
      x = tf.cast(x, dtype)

    return (x - from_val[0]) * (to_val[1] - to_val[0]) / (from_val[1] - from_val[0]) + to_val[0]

  def save_plot(self, examples, epoch, n, write_video=False, no_of_frames=5):
    plt.close()
    
    examples = (examples + 1) / 2.0 # Changing pixel range [-1, 1] to [0, 1]
    
    fig = plt.figure(frameon=False)
    fig.set_size_inches(5.12, 5.12)

    for i in range(n * n):
        plt.subplot(n, n, i+1)
        plt.axis("off")
        plt.imshow(examples[i]) 
    plt.savefig("frame.png", dpi=100)

    videowriter.write_video(path="frame.png", no_of_frames=no_of_frames)

  def plot_loss(d1_loss, d2_loss, g_loss, write_video=False, no_of_frames=5):
    plt.close()

    x = [x for x in range(1, len(d1_loss)+1)]

    fig = plt.figure(frameon=False)
    fig.set_size_inches(5.12, 5.12)

    print(d1_loss)

    plt.plot(x, d1_loss, label='d1_loss', color='b')
    plt.plot(x, d2_loss, label='d2_loss', color='c')
    plt.plot(x, g_loss, label='g_loss', color='g')

    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=5)

    plt.title("Losses")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.show()

    plt.savefig("plot.png", dpi=100)

    videowriter.write_video(path="plot.png", no_of_frames=no_of_frames)

  def load_image(self, path, resize_height=64, resize_width=64):
    image = tf.io.read_file(path)
    image = tf.io.decode_png(image)
    image = tf.image.resize(image, (resize_height, resize_width))
    image = tf.cast(image, tf.float32)
    image = (image-127.5)/127.5
    return image

  def get_dataset(self, path, batch_size=128, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices(path)
    dataset = dataset.shuffle(buffer_size=1024)
    dataset = dataset.map(self.load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    dataset = dataset.batch(batch_size=batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset
      

utils = Utils()

In [ ]:
class VideoWriter():
  def __init__(self):
    self.writer = None

  def create_writer(self, name, res):
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    self.writer = cv2.VideoWriter(name, fourcc, 30.0, res)

  def destroy_writer(self):
    self.writer.release()
    self.writer = None

  def write_video(self, path="frame.png", no_of_frames=1, name='output.avi', res=(512, 512)):
    frame = cv2.imread("frame.png")

    if self.writer == None:
      self.create_writer(name=name, res=res)
      for i in range(no_of_frames):
        self.writer.write(frame)
    else:
      for i in range(no_of_frames):
        self.writer.write(frame)

videowriter = VideoWriter()

In [ ]:
from gan import GAN

ImportError: ignored

In [ ]:
class WGAN(GAN):
    def __init__(self, critic=None, **kwargs):
        super(WGAN, self).__init__(**kwargs)
        self.critic = critic or self.build_critic()
        self.clip_value = 0.01  # Clipping value for the critic weights

    def build_critic(self):
        inputs = tf.keras.layers.Input(shape=(self.img_height, self.img_width, self.img_channels))
        x = self.conv_layer(inputs, 64, 5, batch_norm=False)
        x = self.conv_layer(x, 128, 5)
        x = self.conv_layer(x, 256, 5)
        x = self.conv_layer(x, 512, 5)
        x = tf.keras.layers.Flatten()(x)
        outputs = tf.keras.layers.Dense(1)(x)
        return tf.keras.models.Model(inputs=inputs, outputs=outputs)

    def gradient_penalty(self, batch_size, real_images, fake_images):
        # Generate random interpolation points
        alpha = tf.random.uniform(shape=[batch_size, 1, 1, 1], minval=0., maxval=1.)
        interpolated_images = real_images * alpha + fake_images * (1 - alpha)
        
        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated_images)
            # Compute the critic scores for the interpolated images
            scores = self.critic(interpolated_images)
        # Compute the gradients of the scores w.r.t the interpolated images
        grads = gp_tape.gradient(scores, interpolated_images)
        # Compute the L2-norm of the gradients
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        # Compute the gradient penalty
        gp = tf.reduce_mean(tf.square(norm - 1.))
        return gp

    def train_step(self, real_images):
        # Randomly generate noise for the generator
        batch_size = tf.shape(real_images)[0]
        noise = tf.random.normal(shape=(batch_size, self.latent_dim))

        for _ in range(self.critic_iterations):
            # Train the critic
            with tf.GradientTape() as tape:
                # Generate fake images from the noise
                fake_images = self.generator(noise)
                # Compute the scores of the real and fake images
                real_scores = self.critic(real_images)
                fake_scores = self.critic(fake_images)
                # Compute the critic loss
                critic_loss = fake_scores - real_scores
                # Compute the gradient penalty
                gp = self.gradient_penalty(batch_size, real_images, fake_images)
                critic_loss += self.gp_weight * gp
            # Compute the gradients and update the critic weights
            critic_gradients = tape.gradient(critic_loss, self.critic.trainable_variables)
            self.critic_optimizer.apply_gradients(zip(critic_gradients, self.critic.trainable_variables))
            # Clip the critic weights
            for variable in self.critic.trainable_variables:
                variable.assign(tf.clip_by_value(variable, -self.clip_value, self.clip_value))

        # Train the generator
        with tf.GradientTape() as tape:
            # Generate fake images from the noise
            fake_images = self.generator(noise)
            # Compute the scores of the fake images
            fake_scores = self.critic(fake_images)
            # Compute the generator loss
            generator_loss = -fake_scores
        # Compute the gradients and update the generator weights
        generator_gradients = tape.gradient(generator_loss, self.generator.trainable_variables)
        self.generator_optimizer

In [ ]:
IMAGE_HEIGHT = 64
IMAGE_WIDTH = 64
IMAGE_COLOR = 3

In [ ]:
images_path = glob("/content/data/*")

In [ ]:
dataset = utils.get_dataset(images_path)

In [ ]:
# wgan = WGAN()
wgan = WGAN(
    img_height=64,
    img_width=64,
    img_channels=3,
    latent_dim=100,
    generator_lr=0.0002,
    critic_lr=0.0002,
    gp_weight=10,
    critic_iterations=5
)


In [ ]:
wgan.compile(wgan.d_optimizer, wgan.g_optimizer, wgan.loss_fn)

In [ ]:
for epoch in range(100):
  wgan.fit(dataset, epochs=1)

  n_samples = 25
  noise = np.random.normal(size=(n_samples, wgan.latent_dim))
  examples = wgan.generator.predict(noise)
  utils.save_plot(examples, epoch, int(np.sqrt(n_samples)))
  # utils.plot_loss(d1_loss=gan.d1_loss_list, d2_loss=gan.d2_loss_list, g_loss=gan.g_loss_list, write_video=True)

In [ ]:
for epoch in range(100):
  x = wgan.fit(dataset, epochs=1)
  print(x)

In [ ]:
wgan.generator.save('generator.h5')
wgan.discriminator.save('discriminator.h5')

In [ ]:
  x.history.keys()